# Imports/Installations

In [5]:
import requests
import pandas as pd
import numpy as np
import random
!pip install geopy
from geopy.geocoders import Nominatim
from IPython.display import Image
from IPython.core.display import HTML
from pandas.io.json import json_normalize
!pip install folium
import folium

     |████████████████████████████████| 93 kB 2.8 MB/s eta 0:00:011


# Get lat/lng for Dallas

In [6]:
address = 'Dallas, TX'
geolocator=Nominatim(user_agent="foursquare_agent")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print(latitude, longitude)

32.7762719 -96.7968559


# Define Foursquare credentials/parameters

In [119]:
CLIENT_ID='ENZGOFBBD1EP1YXI0KNP5IDKZHWDOWGZI2KJYNVWQXRRM4R2'
CLIENT_SECRET='ZVM1HRZFPLYIFBILQP4C1A3ZC0ISCD2G3UV5YQXPFFYQBY3A'
VERSION='20201201'
ACCESS_TOKEN='ITL3P30Z55DMIISGOUTDBTLJ0QDHN0VH4D4BKILD0OUFGGLD'
radius=5000
LIMIT=100

# Create url to explore Dallas

In [120]:
url='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

# Get Dallas results

In [121]:
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fed02cd87013231d6928803'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Dallas',
  'headerFullLocation': 'Dallas',
  'headerLocationGranularity': 'city',
  'totalResults': 242,
  'suggestedBounds': {'ne': {'lat': 32.82127194500004,
    'lng': -96.7434347386614},
   'sw': {'lat': 32.73127185499995, 'lng': -96.85027706133859}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bc3321adce4eee1287c719d',
       'name': 'The Joule',
       'location': {'address': '1530 Main St',
        'crossStreet': 'btwn Akard & Ervay St',
        'lat': 32.7807089,
        'lng': -96.7984434,
        'labeledLatLngs': [{

# Focus on the relevant results from the Dallas call

In [122]:
venues=results['response']['groups'][0]['items']

# Get the relevant results in a dataframe

In [123]:
dataframe=json_normalize(venues)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


# Filter the dataframe's columns

In [124]:
dataframe.columns
filtered_columns=['venue.id', 'venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng', 'venue.location.city', 'venue.categories']
df_filt=dataframe.loc[:, filtered_columns]
df_filt

,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.city,venue.categories
0,4bc3321adce4eee1287c719d,The Joule,1530 Main St,32.780709,-96.798443,Dallas,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H..."
1,53b32f5a498e67235a7a828f,Green Door Public House,600 S Harwood St,32.778527,-96.792077,Dallas,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A..."
2,586128b08f0be47e424bb38a,Bread Zeppelin,1300 Main St,32.780309,-96.800749,Dallas,"[{'id': '4bf58dd8d48988d1bd941735', 'name': 'S..."
3,52051fec93cd21d0b480b44d,CBD Provisions,1530 Main St,32.780745,-96.798429,Dallas,"[{'id': '4bf58dd8d48988d157941735', 'name': 'N..."
4,4b097e7ff964a5203b1823e3,Main Street Garden,1900 Main St,32.781136,-96.795183,Dallas,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P..."
...,...,...,...,...,...,...,...
95,4b8c16dcf964a52048bd32e3,Stanley Korshak,500 Crescent Ct Ste 100,32.794864,-96.804485,Dallas,"[{'id': '4bf58dd8d48988d104951735', 'name': 'B..."
96,4abe34d4f964a520f48b20e3,"Le Méridien Dallas, The Stoneleigh",2927 Maple Ave,32.797745,-96.807282,Dallas,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H..."
97,4d115af70c786ea8994bd0ab,Nick & Sam's,3008 Maple Ave,32.798509,-96.807502,Dallas,"[{'id': '4bf58dd8d48988d1cc941735', 'name': 'S..."
98,50abfbfde4b063c5d67726c9,Craft and Growler,3601 Parry Avenue,32.781557,-96.766893,Dallas,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B..."


# Define a function to get a "cleaner" category value

In [125]:
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']

# Clean up the columns and apply our category function from above

In [126]:
df_filt.columns = [col.split('.')[-1] for col in df_filt.columns]
df_filt['categories']=df_filt.apply(get_category_type, axis=1)

In [128]:
#make it easier to see the whole dataframe when it is called
pd.set_option('display.max_rows', 105)
df_filt

,id,name,address,lat,lng,city,categories
0,4bc3321adce4eee1287c719d,The Joule,1530 Main St,32.780709,-96.798443,Dallas,Hotel
1,53b32f5a498e67235a7a828f,Green Door Public House,600 S Harwood St,32.778527,-96.792077,Dallas,American Restaurant
2,586128b08f0be47e424bb38a,Bread Zeppelin,1300 Main St,32.780309,-96.800749,Dallas,Salad Place
3,52051fec93cd21d0b480b44d,CBD Provisions,1530 Main St,32.780745,-96.798429,Dallas,New American Restaurant
4,4b097e7ff964a5203b1823e3,Main Street Garden,1900 Main St,32.781136,-96.795183,Dallas,Park
5,4bbb66813db7b713c949249a,Pioneer Plaza,530 S Griffin St,32.776616,-96.801539,Dallas,Plaza
6,4d73f2df9a296ea8cf424fa9,"The Statler, Curio Collection by Hilton",1914 Commerce St,32.780469,-96.794641,Dallas,Hotel
7,4eaf21a68b81d80bd7a1922b,Omni Dallas Hotel,555 S Lamar St,32.775332,-96.804357,Dallas,Hotel
8,56b61d3f498e1798d42f6f4b,Alamo Drafthouse Cinema,1005 South Lamar,32.769801,-96.799298,Dallas,Movie Theater
9,56a17841498ecca51c3a96f1,The Westin Dallas Downtown,1201 Main St,32.780357,-96.802063,Dallas,Hotel


# Extract the venues that have "Bar" in the category

In [146]:
bars=df_filt.categories.str.contains('Bar', case=False)
df_filt[bars]

,id,name,address,lat,lng,city,categories
10,57cdc344498e5242ef29a53a,The Crafty Irishman,1800 Main St Ste102,32.7813,-96.7962,Dallas,Bar
11,5a47374d646e381ec6e07a28,Waterproof,1924 Commerce,32.7811,-96.7952,Dallas,Hotel Bar
22,40e0b100f964a520c3081fe3,Lee Harvey's,1807 Gould St,32.7681,-96.7895,Dallas,Dive Bar
30,58058a4c38faac6949f92d0e,Dot's Hop House,2645 Commerce St,32.7832,-96.7845,Dallas,Bar
43,4bb7daa398c7ef3b097b3002,Black Swan Saloon,2708 Elm St,32.7844,-96.7843,Dallas,Dive Bar
44,4e723125ae6067986ece5b9c,Uncle Uber's,2713 Commerce St,32.7833,-96.7834,Dallas,Sports Bar
57,5ae1569fb9b37b002cf2021d,Ruins,2653 Commerce Street,32.7832,-96.7841,Dallas,Cocktail Bar
66,561595ae498edfd54b5f1c5e,Happiest Hour,2616 Olive St,32.7911,-96.8068,Dallas,Bar
83,527ec3d111d25111aa2b317a,Eighteen/76,NaN,32.7913,-96.8102,Dallas,Bar
84,577c74ca38fa745171b35c4d,The Tipsy Alchemist,2101 Cedar Springs Road,32.7941,-96.8063,Dallas,Cocktail Bar


# Plot the venues, and labels with their name/category, onto a map of Dallas

In [187]:
map_dallas=folium.Map(location=[latitude, longitude], zoom_start=14)
for lat, lng, name, category in zip(df_filt[bars].lat, df_filt[bars].lng, df_filt[bars].name, df_filt[bars].categories):
    label=folium.Popup(name + ', ' + category, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fillcolor='#3186cc', fill_opacity=0.7, parse_html=False).add_to(map_dallas)
map_dallas

# Get the lat/lng for Boise

In [149]:
address2 = 'Boise, ID'
geolocator=Nominatim(user_agent="foursquare_agent")
location2=geolocator.geocode(address2)
latitude2=location2.latitude
longitude2=location2.longitude
print(latitude2, longitude2)

43.6166163 -116.200886


# Create a url to explore Boise and get a JSON of the results

In [151]:
url2='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude2, longitude2, VERSION, radius, LIMIT)
results2=requests.get(url2).json()
results2

{'meta': {'code': 200, 'requestId': '5fed0ab90062d40d7ec62703'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Boise',
  'headerFullLocation': 'Boise',
  'headerLocationGranularity': 'city',
  'totalResults': 142,
  'suggestedBounds': {'ne': {'lat': 43.66161634500004,
    'lng': -116.13884487824001},
   'sw': {'lat': 43.57161625499995, 'lng': -116.26292712175999}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d4b730305556ea8d862a828',
       'name': 'Fork',
       'location': {'address': '199 N 8th St',
        'crossStreet': 'Idaho',
        'lat': 43.61639983692314,
        'lng': -116.20274347848748,
        'labeledLatLngs': [{'labe

# Get the relevant data from the results

In [152]:
venues2=results2['response']['groups'][0]['items']

# Get the relevant data into a dataframe

In [153]:
dataframe2=json_normalize(venues2)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


# Filter the dataframe down to the relevant columns only

In [154]:
dataframe2.columns
filtered_columns2=['venue.id', 'venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng', 'venue.location.city', 'venue.categories']
df_filt2=dataframe2.loc[:, filtered_columns2]
df_filt2

,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.city,venue.categories
0,4d4b730305556ea8d862a828,Fork,199 N 8th St,43.616400,-116.202743,Boise,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A..."
1,4bad27aaf964a5207e333be3,Bittercreek Ale House,246 N 8th St,43.616612,-116.202501,Boise,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B..."
2,50e4c1f3e4b08ec8f660dfe1,Àlavita,807 W Idaho St,43.616636,-116.203150,Boise,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I..."
3,514a3386e4b034cf090ebfba,10 Barrel Brewing,830 W Bannock St,43.617764,-116.202886,Boise,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B..."
4,4b48c8d6f964a520d15626e3,Flying M Coffeehouse,500 W Idaho St,43.614809,-116.199321,Boise,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C..."
5,4a649172f964a52045c61fe3,Goldy's Breakfast Bistro,108 S Capitol Blvd,43.614733,-116.202712,Boise,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B..."
6,4b59f5a7f964a520d9a328e3,Taj Mahal,150 N 8th St Ste 222,43.616029,-116.203112,Boise,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I..."
7,4dd3ef1eb0fbf653b6487ea7,John Berryhill's Bacon,915 W Idaho St.,43.617228,-116.204756,Boise,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C..."
8,4e20e1d17d8b71715ba5c456,Freak Alley,NaN,43.617381,-116.203269,NaN,"[{'id': '4bf58dd8d48988d1e2931735', 'name': 'A..."
9,4b50bfe6f964a520b83027e3,Front Door,105 S 6th St,43.614271,-116.201492,Boise,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P..."


# Clean up the columns

In [159]:
df_filt2.columns = [col.split('.')[-1] for col in df_filt2.columns]
df_filt2

,id,name,address,lat,lng,city,categories
0,4d4b730305556ea8d862a828,Fork,199 N 8th St,43.616400,-116.202743,Boise,American Restaurant
1,4bad27aaf964a5207e333be3,Bittercreek Ale House,246 N 8th St,43.616612,-116.202501,Boise,Bar
2,50e4c1f3e4b08ec8f660dfe1,Àlavita,807 W Idaho St,43.616636,-116.203150,Boise,Italian Restaurant
3,514a3386e4b034cf090ebfba,10 Barrel Brewing,830 W Bannock St,43.617764,-116.202886,Boise,Brewery
4,4b48c8d6f964a520d15626e3,Flying M Coffeehouse,500 W Idaho St,43.614809,-116.199321,Boise,Coffee Shop
5,4a649172f964a52045c61fe3,Goldy's Breakfast Bistro,108 S Capitol Blvd,43.614733,-116.202712,Boise,Breakfast Spot
6,4b59f5a7f964a520d9a328e3,Taj Mahal,150 N 8th St Ste 222,43.616029,-116.203112,Boise,Indian Restaurant
7,4dd3ef1eb0fbf653b6487ea7,John Berryhill's Bacon,915 W Idaho St.,43.617228,-116.204756,Boise,Café
8,4e20e1d17d8b71715ba5c456,Freak Alley,NaN,43.617381,-116.203269,NaN,Art Gallery
9,4b50bfe6f964a520b83027e3,Front Door,105 S 6th St,43.614271,-116.201492,Boise,Pizza Place


# Extract the venues that have "Bar" in the category

In [161]:
bars2=df_filt2.categories.str.contains('Bar', case=False)
df_filt2[bars2]

,id,name,address,lat,lng,city,categories
1,4bad27aaf964a5207e333be3,Bittercreek Ale House,246 N 8th St,43.616612,-116.202501,Boise,Bar
10,4f061f2177c8f87cd21c87f3,Taphouse,760 W Main St,43.615330,-116.203014,Boise,Sports Bar
18,4b47ca74f964a520dc3e26e3,Pengilly's Saloon,513 W Main St,43.614046,-116.200770,Boise,Bar
20,4b496efaf964a520bd6f26e3,Modern Hotel & Bar,1314 W Grove St,43.618333,-116.210421,Boise,Hotel Bar
69,4b775a31f964a5205f942ee3,Little Dutch Garden,1910 S Owyhee St,43.585793,-116.223124,Boise,Dive Bar
85,4b4e1f90f964a5200ae226e3,The Lift Bar & Grill,4091 W State St,43.642890,-116.239957,Boise,Bar
92,4be63520cf200f475363143c,Crescent Bar & Grill,5500 W Franklin Rd,43.603850,-116.250230,Boise,Bar
96,4b4a6188f964a520f48526e3,The Dutch Goose,3515 W State St,43.637999,-116.233415,Boise,Dive Bar


# Plot the bars, along with labels showing their name and category, onto a map of Boise

In [194]:
map_boise=folium.Map(location=[latitude2, longitude2], zoom_start=13)
for lat, lng, name, category in zip(df_filt2[bars2].lat, df_filt2[bars2].lng, df_filt2[bars2].name, df_filt2[bars2].categories):
    label=folium.Popup(name + ', ' + category, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fillcolor='#3186cc', fill_opacity=0.7, parse_html=False).add_to(map_boise)
map_boise